# Boilerplate notebook

In [ ]:
# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Pillow
from PIL import Image
# Torch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms

# 1. Download dataset

In [ ]:
!git clone -b data https://github.com/Oxiang/50.039-Deep-Learning.git

In [ ]:
!sudo apt-get install tree

In [ ]:
cd 50.039-Deep-Learning

In [ ]:
%%bash

(
tree dataset -d
) 

# 2. Dataset high-level info

The images stored in the **./dataset** folder and its subfolder consists of 150 by 150 pixels greyscale images, representing X-Ray pictures of lungs.

The images, consists of X-ray pictures of the following:

| Description                              | Class index | Tensor  | Class label        |
| ---------------------------------------- | ----------- | ------- | ------------------ |
| People with no infection diagnosis       | 0           | [1 0 0] | normal             |
| People with infected lungs and non-covid | 1           | [0 1 0] | infected_non_covid |
| People with infected lungs and covid     | 2           | [0 0 1] | infected_covid     |



In [ ]:
classes = {0: 'normal', 1: 'infected_non_covid', 2: 'infected_covid'}
groups = ['train', 'test', 'val']
dataset_numbers = {
    'train_normal': 1341,
    'train_infected_non_covid': 2530,
    'train_infected_covid': 1345,
    'val_normal': 8,
    'val_infected_non_covid': 8,
    'val_infected_covid': 8,    
    'test_normal': 234,
    'test_infected_non_covid': 242,
    'test_infected_covid': 138,
}
dataset_paths = {
    'train_normal': './dataset/train/normal/',
    'train_infected_non_covid': './dataset/train/infected/non-covid/',
    'train_infected_covid': './dataset/train/infected/covid/',
    'val_normal': './dataset/val/normal/',
    'val_infected_non_covid': './dataset/val/infected/non-covid/',
    'val_infected_covid': './dataset/val/infected/covid/',    
    'test_normal': './dataset/test/normal/',
    'test_infected_non_covid': './dataset/test/infected/non-covid/',
    'test_infected_covid': './dataset/test/infected/covid/',    
}

View one of the images and its properties. These images consist of a Numpy array, with values ranging between 0 and 255. These values will be normalized.

In [ ]:
path_to_file = './dataset/train/normal/1.jpg'
with open(path_to_file, 'rb') as f:
    im = np.asarray(Image.open(f))
    plt.imshow(im)
f.close()
print('Image shape is: {}'.format(im.shape))
# Images are defined as a Numpy array of values between 0 and 256
print('Image as a numpy array is:\n {}'.format(im))